In [12]:
import datasetmaker
import tifffile
import numpy as np
from tqdm import tqdm
from pathlib import Path
import os
import pandas as pd
import sys
# Get the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the parent directory to sys.path
sys.path.append(parent_dir)
from onlypores import onlypores as op
from UTXCTregister import UTXCTregister as reg

In [13]:
def read_sequence(folder_path):
    """
    Read a sequence of TIFF files in a folder as a 3D volume.
    
    Args:
    folder_path (str): Path to the folder containing TIFF files.

    Returns:
    numpy.ndarray: A 3D array where each slice corresponds to a TIFF file.
    """

    # List and sort the TIFF files
    tiff_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if (f.endswith('.tiff') or f.endswith('.tif'))])

    tiff_sequence = tifffile.TiffSequence(tiff_files)
    
    # Get the total number of TIFF files
    total_files = len(tiff_files)
    
    # Read each TIFF file and update progress
    volume = []
    with tqdm(total=total_files, desc="Progress") as pbar:
        for i, file_path in enumerate(tiff_files):
            slice_data = tifffile.imread(file_path)
            volume.append(slice_data)
            
            # Update progress
            pbar.update(1)
    
    return np.array(volume)

def write_sequence(folder_path, name, volume):
    """
    Save a 3D volume as a sequence of TIFF files in a folder.
    
    Args:
    folder_path (str): Path to the folder where TIFF files will be saved.
    name (str): Name of the TIFF files.
    volume (numpy.ndarray): A 3D array where each slice corresponds to an image.
    """

    folder_path = folder_path / name

    # Create the folder if it doesn't exist
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    # Save each slice as a TIFF file with progress bar
    with tqdm(total=volume.shape[0], desc="Saving") as pbar:
        for i in range(volume.shape[0]):
            tifffile.imwrite(f"{folder_path}/{name}_{i:04d}.tif", volume[i])
            pbar.update(1)
    
    print("Saving complete.")

def to_matrix(string):

    matrix1 = float(string[2:17])

    matrix2 = float(string[17:33])

    matrix3 = float(string[33:49])

    matrix4 = float(string[53:68])

    matrix5 = float(string[68:84])

    matrix6 = float(string[84:100])

    matrix7 = float(string[105:121])

    matrix8 = float(string[121:137])

    matrix9 = float(string[137:151])

    matrix = np.array([[matrix1,matrix2,matrix3],[matrix4,matrix5,matrix6],[matrix7,matrix8,matrix9]])

    return matrix
        

In [14]:
database = pd.read_csv(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\11_database\Database.csv')

database = database.replace({np.nan: None})

In [15]:
database.columns

Index(['Id', 'Original', 'Eq', 'Frontal 90 Right', 'XCT padded', 'Binary',
       'Material Mask', 'Onlypores', 'UT', 'UT aligned', 'UT aligned amp',
       'UT registered', 'UT lente', 'UT aligned lente', 'UT aligned amp lente',
       'UT registered lente', 'Registration Parameters',
       'Registration Parameters lente', 'Dataset PatchvsVolfrac 3x3',
       'Dataset PatchvsVolfrac lente 3x3', 'Dataset PatchvsVolfrac 5x5',
       'Dataset PatchvsVolfrac lente 5x5', 'Dataset PatchvsVolfrac 7x7',
       'Dataset PatchvsVolfrac lente 7x7'],
      dtype='object')

In [16]:
DatasetsFolder = Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\04_ML_data\Airbus\Panel Pegaso')

In [17]:
ids = database['Id']

onlypores_paths = database['Onlypores']

masks = database['Material Mask']

datasets1x3 = database['Dataset PatchvsVolfrac 3x3']

datasets1x5 = database['Dataset PatchvsVolfrac 5x5']

datasets1x7 = database['Dataset PatchvsVolfrac 7x7']

datasets1x3_lente = database['Dataset PatchvsVolfrac lente 3x3']

datasets1x5_lente = database['Dataset PatchvsVolfrac lente 5x5']

datasets1x7_lente = database['Dataset PatchvsVolfrac lente 7x7']

rfs = database['UT registered']

rfs_lente = database['UT registered lente']

rfs_og = database['UT']

rfs_lente_og = database['UT lente']

registration_parameters = database['Registration Parameters']

registration_parameters_lente = database['Registration Parameters lente']

frontals  = database['Frontal 90 Right']

In [18]:
datasets1x3[datasets1x3 != 0] = None

C:\Users\alberto.vicente\AppData\Local\Temp\ipykernel_10276\1479614042.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasets1x3[datasets1x3 != 0] = None


In [19]:
# for i in range(len(ids)):

#     if (onlypores_paths[i] != None) and (masks[i] != None) and (rfs[i] != None):
#         print(f"Processing {ids[i]}")
#         try:
#             #Check if is there a remaining dataset to be processed
#             if(datasets1x3[i]==None) or (datasets2x3[i]==None) or (datasets1x5[i]==None) or (datasets2x5[i]==None) or (datasets1x7[i]==None) or (datasets2x7[i]==None):

#                 print('Loading:',onlypores_paths[i])
#                 onlypores = read_sequence(onlypores_paths[i])
#                 print('Loading:',masks[i])
#                 mask = read_sequence(masks[i])
#                 print('Loading:',rfs[i])
#                 rf = tifffile.imread(rfs[i])

#                 output_folder = DatasetsFolder / str(ids[i])

#                 if output_folder.exists() == False:
#                     output_folder.mkdir()
                
#                 output_folder = output_folder / 'MonoElement'

#                 if output_folder.exists() == False:
#                     output_folder.mkdir()

#                 if (datasets1x3[i]==None) or (datasets2x3[i]==None):
#                     print('Processing 3x3')
#                     datasetmaker.main(onlypores,mask,rf,output_folder,ut_patch_size=3)
#                     datasets1x3[i] = str(output_folder / 'patch_vs_volfrac_3.csv')
#                     datasets2x3[i] = str(output_folder / 'patch_vs_patch_3.csv')
#                 if (datasets1x5[i]==None) or (datasets2x5[i]==None):
#                     print('Processing 5x5')
#                     datasetmaker.main(onlypores,mask,rf,output_folder,ut_patch_size=5)
#                     datasets1x5[i] = str(output_folder / 'patch_vs_volfrac_5.csv')
#                     datasets2x5[i] = str(output_folder / 'patch_vs_patch_5.csv')
#                 if (datasets1x7[i]==None) or (datasets2x7[i]==None):
#                     print('Processing 7x7')
#                     datasetmaker.main(onlypores,mask,rf,output_folder,ut_patch_size=7)
#                     datasets1x7[i] = str(output_folder / 'patch_vs_volfrac_7.csv')
#                     datasets2x7[i] = str(output_folder / 'patch_vs_patch_7.csv')
            
#             else:
#                 print('All datasets already processed')
        
#         except Exception as e:
#             print(f"Error processing {ids[i]}")
#             print(e)
    
#     if (onlypores_paths[i] != None) and (masks[i] != None) and (rfs_lente[i] != None):
#         print(f"Processing {ids[i]} lente")
#         try:
#             #Check if is there a remaining dataset to be processed
#             if(datasets1x3_lente[i]==None) or (datasets2x3_lente[i]==None) or (datasets1x5_lente[i]==None) or (datasets2x5_lente[i]==None) or (datasets1x7_lente[i]==None) or (datasets2x7_lente[i]==None):

#                 print('Loading:',onlypores_paths[i])
#                 onlypores = read_sequence(onlypores_paths[i])
#                 print('Loading:',masks[i])
#                 mask = read_sequence(masks[i])
#                 print('Loading:',rfs_lente[i])
#                 rf = tifffile.imread(rfs_lente[i])

#                 output_folder = DatasetsFolder / str(ids[i])

#                 if output_folder.exists() == False:
#                     output_folder.mkdir()

#                 output_folder = output_folder / 'MonoElementLens'

#                 if output_folder.exists() == False:
#                     output_folder.mkdir()

#                 if (datasets1x3_lente[i]==None) or (datasets2x3_lente[i]==None):
#                     print('Processing 3x3')
#                     datasetmaker.main(onlypores,mask,rf,output_folder,ut_patch_size=3)
#                     datasets1x3_lente[i] = str(output_folder / 'patch_vs_volfrac_3.csv')
#                     datasets2x3_lente[i] = str(output_folder / 'patch_vs_patch_3.csv')
#                 if (datasets1x5_lente[i]==None) or (datasets2x5_lente[i]==None):
#                     print('Processing 5x5')
#                     datasetmaker.main(onlypores,mask,rf,output_folder,ut_patch_size=5)
#                     datasets1x5_lente[i] = str(output_folder / 'patch_vs_volfrac_5.csv')
#                     datasets2x5_lente[i] = str(output_folder / 'patch_vs_patch_5.csv')
#                 if (datasets1x7_lente[i]==None) or (datasets2x7_lente[i]==None):
#                     print('Processing 7x7')
#                     datasetmaker.main(onlypores,mask,rf,output_folder,ut_patch_size=7)
#                     datasets1x7_lente[i] = str(output_folder / 'patch_vs_volfrac_7.csv')
#                     datasets2x7_lente[i] = str(output_folder / 'patch_vs_patch_7.csv')
            
#             else:
#                 print('All datasets already processed')
        
#         except Exception as e:
#             print(f"Error processing {ids[i]}")
#             print(e)


In [ ]:
for i in range(len(ids)):

    if (onlypores_paths[i] != None) and (masks[i] != None) and (rfs[i] != None):
        print(f"Processing {ids[i]}")
        try:
            #Check if is there a remaining dataset to be processed
            if(datasets1x3[i]==None) or (datasets1x5[i]==None) or (datasets1x7[i]==None):
                
                print('Loading xct')
                #get onlypores
                xct = tifffile.imread(frontals[i])
                xct = np.swapaxes(xct, 0, 1)
                xct = np.swapaxes(xct, 1, 2)
                print('Loading rf')
                rf = tifffile.imread(rfs_og[i])
                rf = np.swapaxes(rf, 0, 1)
                rf = np.swapaxes(rf, 1, 2)
                print('Aplying registration')
                parameters = to_matrix(registration_parameters[i])
                xct = reg.apply_registration(rf,xct,parameters)
                rf = np.swapaxes(rf, 1, 2)
                rf = np.swapaxes(rf, 0, 1)
                xct = np.swapaxes(xct, 1, 2)
                xct = np.swapaxes(xct, 0, 1)
                print('Generating onlypores')
                onlypores, mask, _ = op.onlypores_parallel(xct)
                print('Onlypores generated')

                output_folder = DatasetsFolder / str(ids[i])

                if output_folder.exists() == False:
                    output_folder.mkdir()
                
                output_folder = output_folder / 'MonoElement'

                if output_folder.exists() == False:
                    output_folder.mkdir()

                if (datasets1x3[i]==None) :
                    print('Processing 3x3')
                    datasetmaker.main(onlypores,mask,rf,output_folder,ut_patch_size=3)
                    datasets1x3[i] = str(output_folder / 'patch_vs_volfrac_3.csv')
                if (datasets1x5[i]==None) :
                    print('Processing 5x5')
                    datasetmaker.main(onlypores,mask,rf,output_folder,ut_patch_size=5)
                    datasets1x5[i] = str(output_folder / 'patch_vs_volfrac_5.csv')
                if (datasets1x7[i]==None) :
                    print('Processing 7x7')
                    datasetmaker.main(onlypores,mask,rf,output_folder,ut_patch_size=7)
                    datasets1x7[i] = str(output_folder / 'patch_vs_volfrac_7.csv')
            
            else:
                print('All datasets already processed')
        
        except Exception as e:
            print(f"Error processing {ids[i]}")
            print(e)
    
    if (onlypores_paths[i] != None) and (masks[i] != None) and (rfs_lente[i] != None):
        print(f"Processing {ids[i]} lente")
        try:
            #Check if is there a remaining dataset to be processed
            if(datasets1x3_lente[i]==None) or (datasets1x5_lente[i]==None) or (datasets1x7_lente[i]==None):

                
                print('Loading rf')
                rf = tifffile.imread(rfs_lente_og[i])
                rf = np.swapaxes(rf, 0, 1)
                rf = np.swapaxes(rf, 1, 2)
                print('Aplying registration')
                parameters = to_matrix(registration_parameters[i])
                xct = reg.apply_registration(rf,xct,parameters)
                rf = np.swapaxes(rf, 1, 2)
                rf = np.swapaxes(rf, 0, 1)
                xct = np.swapaxes(xct, 1, 2)
                xct = np.swapaxes(xct, 0, 1)
                print('Generating onlypores')
                onlypores, mask, _ = op.onlypores_parallel(xct)
                print('Onlypores generated')

                output_folder = DatasetsFolder / str(ids[i])

                if output_folder.exists() == False:
                    output_folder.mkdir()

                output_folder = output_folder / 'MonoElementLens'

                if output_folder.exists() == False:
                    output_folder.mkdir()

                if (datasets1x3_lente[i]==None) :
                    print('Processing 3x3')
                    datasetmaker.main(onlypores,mask,rf,output_folder,ut_patch_size=3)
                    datasets1x3_lente[i] = str(output_folder / 'patch_vs_volfrac_3.csv')
                if (datasets1x5_lente[i]==None) :
                    print('Processing 5x5')
                    datasetmaker.main(onlypores,mask,rf,output_folder,ut_patch_size=5)
                    datasets1x5_lente[i] = str(output_folder / 'patch_vs_volfrac_5.csv')
                if (datasets1x7_lente[i]==None) :
                    print('Processing 7x7')
                    datasetmaker.main(onlypores,mask,rf,output_folder,ut_patch_size=7)
                    datasets1x7_lente[i] = str(output_folder / 'patch_vs_volfrac_7.csv')
            
            else:
                print('All datasets already processed')
        
        except Exception as e:
            print(f"Error processing {ids[i]}")
            print(e)


Processing 1_26
Loading xct


In [10]:
database['Dataset PatchvsVolfrac 3x3'] = datasets1x3

database['Dataset PatchvsVolfrac 5x5'] = datasets1x5

database['Dataset PatchvsVolfrac 7x7'] = datasets1x7

database['Dataset PatchvsVolfrac lente 3x3'] = datasets1x3_lente

database['Dataset PatchvsVolfrac lente 5x5'] = datasets1x5_lente

database['Dataset PatchvsVolfrac lente 7x7'] = datasets1x7_lente

In [11]:
database.to_csv(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\11_database\Database.csv', index=False)